# Part 1

In [1]:
filename = "inputs/12-16.txt"
with open(filename, "r") as f:
    data = f.read()

In [2]:
import re
import networkx as nx

In [3]:
flows = {}
G = nx.Graph()
for l in data.splitlines():
    m = re.match("Valve ([A-Z]+) has flow rate=(\d+); tunnels? leads? to valves? (.*)", l)
    valve, flow, neighbors = m.groups()
    for n in neighbors.split(", "):
        G.add_edge(valve, n)
    flows[valve] = int(flow)

In [4]:
flows = {v : flows[v] for v in flows if flows[v] or v == "AA"}
distances = {(x, y): nx.shortest_path_length(G, x, y) for x in flows for y in flows}

In [5]:
def pressure(t, valves, pos, distances, flows):
    if len(valves) == 0 or t < 0:
        return 0
    max_flow = 0
    for v in valves:
        nt = t - distances[pos, v] - 1
        flow = flows[v] * nt
        if flow <= 0:
            continue
        max_flow = max(max_flow, flow + pressure(nt, valves - {v}, v, distances, flows))
    return max_flow

In [6]:
pressure(30, set(flows.keys()), "AA", distances, flows)

1880

# Part 2

In [7]:
from functools import cache

In [8]:
@cache
def pressure_human(t, valves, pos):
    if len(valves) == 0 or t < 0:
        return 0
    max_flow = 0
    for v in valves:
        nt = t - distances[pos, v] - 1
        flow = flows[v] * nt
        if flow <= 0:
            continue
        max_flow = max(max_flow, flow + pressure_human(nt, valves - {v}, v))
    return max_flow

In [9]:
@cache
def pressure_elephant(t, valves, pos):
    if len(valves) == 0 or t < 0:
        return pressure_human(26, valves, "AA")
    max_flow = 0
    for v in valves:
        nt = t - distances[pos, v] - 1
        flow = flows[v] * nt
        if flow <= 0:
            continue
        max_flow = max([max_flow, flow + pressure_elephant(nt, valves - {v}, v), flow + pressure_human(26, valves - {v}, "AA")])
    return max_flow

In [10]:
pressure_elephant(26, frozenset(flows.keys()), "AA")

2520